In [1]:
import itertools

from utils import (
    DatasetConfig,
    DatasetTypeEnum,
    DatasetClassification,
    DataLoader,
    get_pretrained_model,
    get_cross_entropy_loss,
    get_sgd_optimizer,
    CNNParams,
    train_one_epoch,
    val_one_epoch,
)
from utils import *
from .classification import get_cnn_params_1, get_cnn_params_3

import pandas as pd

In [2]:
cnn_models = [
    get_cnn_params_1(),
    get_cnn_params_3(),
]
kfolds = 5
models_path = "./cls_models"


In [3]:
print(cnn_models)

[CNNHyperParams(dataset_file='resisc45_dataset.csv', model_name=<PretrainedModelsEnum.resnet18: 'resnet18'>, num_classes=45, feature_extract=True, batch_size=32, num_epochs=20, criterion_name='cross_entropy', optimizer_name='sgd', train_transforms=Compose(
    RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=bilinear, antialias=warn)
    RandomHorizontalFlip(p=0.5)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
), val_transforms=Compose(
    Resize(size=256, interpolation=bilinear, max_size=None, antialias=warn)
    CenterCrop(size=(224, 224))
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
), use_pretrained=True), CNNHyperParams(dataset_file='ucmercedlu_dataset.csv', model_name=<PretrainedModelsEnum.resnet18: 'resnet18'>, num_classes=21, feature_extract=True, batch_size=32, num_epochs=20, criterion_name='cross_entropy', optimizer_name='sgd', train_transforms=Compose(
    R

In [4]:
def train_val_dataloaders(hyperparams: CNNParams, val_fold: int):
    train_config = DatasetConfig(
        dataset_file=hyperparams.dataset_file,
        transform=hyperparams.train_transforms,
        dataset_type=DatasetTypeEnum.train,
        val_fold=val_fold,
    )
    val_config = DatasetConfig(
        dataset_file=hyperparams.dataset_file,
        transform=hyperparams.val_transforms,
        dataset_type=DatasetTypeEnum.val,
        val_fold=val_fold,
    )

    train_set = DatasetClassification(train_config)
    val_set = DatasetClassification(val_config)

    train_dataloader = DataLoader(train_set, batch_size=hyperparams.batch_size, shuffle=True)
    val_dataloader = DataLoader(val_set, batch_size=hyperparams.batch_size, shuffle=False)

    return train_dataloader, val_dataloader


In [5]:
def train_model_on_fold(hyperparams: CNNParams, val_fold: int, save_path: str):
    print(hyperparams.model_name, hyperparams.dataset_file, val_fold, save_path)

    with open(f"{models_path}/model_{hyperparams.dataset_file}_{val_fold}", "w") as f:
        f.write(f"{str(hyperparams)}; {val_fold}; {save_path}")

    train_data, val_data = train_val_dataloaders(hyperparams, val_fold)

    cnn_model, input_size = get_pretrained_model(hyperparams)
    cnn_model.to(get_device())

    if hyperparams.criterion_name == "cross_entropy":
        criterion = get_cross_entropy_loss()
    if hyperparams.optimizer_name == 'sgd':
        optimizer = get_sgd_optimizer(cnn_model, feature_extract=hyperparams.feature_extract, lr=0.001, momentum=0.9)

    train_losses = []
    val_losses = []
    for epoch in range(hyperparams.num_epochs):
        print(f"Epoch {epoch+1}\n-------------------------------")
        train_loss = train_one_epoch(criterion, optimizer, train_data, cnn_model, device=get_device())
        val_loss = val_one_epoch(criterion, val_data, cnn_model, device=get_device())
        train_losses.append(train_loss)
        val_losses.append(val_loss)
    
    torch.save(cnn_model, f"{models_path}/model_{hyperparams.dataset_file}_{val_fold}.pt")
    
    pd.DataFrame({
        'train_losses': train_losses,
        'val_losses': val_losses,
    }).to_csv(f"{models_path}/model_{hyperparams.dataset_file}_{val_fold}_loss.csv")

    print("done.")
    # save best torch model in save_path / dataset_file / model_name / val_fold


In [6]:
for hyperparams, val_fold in itertools.product(cnn_models, range(kfolds)):
    train_model_on_fold(hyperparams, val_fold, save_path=models_path)


PretrainedModelsEnum.resnet18 resisc45_dataset.csv 0 ./cls_models
Using mps device
Params to learn:
	 fc.weight
	 fc.bias
Epoch 1
-------------------------------
Using mps device
Started train.
mps
loss: 3.801620  [   32/31500]
loss: 3.300227  [ 3232/31500]
loss: 2.667263  [ 6432/31500]
loss: 2.273973  [ 9632/31500]
loss: 1.843383  [12832/31500]
loss: 2.121815  [16032/31500]
loss: 1.538135  [19232/31500]
loss: 1.594334  [22432/31500]
loss: 1.856065  [25632/31500]
loss: 1.648572  [28832/31500]
Using mps device
Started validation.
Test Error: 
 Accuracy: 69.9%, Avg loss: 1.221838 

Epoch 2
-------------------------------
Using mps device
Started train.
mps
loss: 1.825070  [   32/31500]
loss: 1.425711  [ 3232/31500]
loss: 1.537689  [ 6432/31500]
loss: 1.143354  [ 9632/31500]
loss: 1.269049  [12832/31500]
loss: 1.208313  [16032/31500]
loss: 1.438406  [19232/31500]
loss: 1.357798  [22432/31500]
loss: 1.122847  [25632/31500]
loss: 0.943316  [28832/31500]
Using mps device
Started validation.
